<head><h1 align="center">
Front End Development
</h1></head>  
  
<head><h3 align="center">Recipe Recommender System Project Notebook</h3></head>  

`Streamlit` development will be contained in a python file `app.py`. This notebook will purely contain notes to and drafts that coincide with development.

#### Setup

**Install Dependencies:**

In [3]:
# %pip install wordcloud
# %pip install surprise

**Import Settings,  
Libraries & Functions:**

In [4]:
# sys.path.insert(0, '/home/ec2-user/SageMaker/recipe-book/code/')
from code.functions import *
import streamlit as st
# from code.aws_connect import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data

In [22]:
rdf = pd.read_csv('data/RAW_recipes.csv')
idf = pd.read_csv('data/RAW_interactions.csv')

## Cleaning & Data Preparation

In [23]:
# Cleaning: Dropping row that contains a NaN value for recipe name
rdf.drop(labels=721, inplace = True)

# Cleaning/FE: Creating columns for recipe's respective nutrients
rdf['kcal'] = rdf.nutrition.apply(lambda x: x[1:-1].split(sep=', ')[0])
rdf['fat'] = rdf.nutrition.apply(lambda x: x[1:-1].split(sep=', ')[1])
rdf['sugar'] = rdf.nutrition.apply(lambda x: x[1:-1].split(sep=', ')[2])
rdf['salt'] = rdf.nutrition.apply(lambda x: x[1:-1].split(sep=', ')[3])
rdf['protein'] = rdf.nutrition.apply(lambda x: x[1:-1].split(sep=', ')[4])
rdf['sat_fat'] = rdf.nutrition.apply(lambda x: x[1:-1].split(sep=', ')[5])
rdf['carbs'] = rdf.nutrition.apply(lambda x: x[1:-1].split(sep=', ')[6])

# Cleaning: Imputing outlier value to median
rdf['minutes'] = np.where(rdf.minutes == 2147483647,
                         rdf.minutes.median(),
                         rdf.minutes)

idf['date'] = pd.to_datetime(idf.date)

# Getting Started

Run the following via command line in Terminal (not AWS Sagemaker instance command line).
> `pip install streamlit`  

After installation runs...  
> `streamlit hello`

# Data Reconstruction

For the app, I will segment the recipes dataset in to different categories, allowing the user to select recipes of different classifications.

## Grouping by Tags and Writing to CSV

### With `tags_to_csv()` function

#### Types of Cuisines:

In [158]:
cuisines = ['french',
            'german',
            'swiss',
            'spanish',
            'greek',
            'ethiopian', 
            'korean',
            'chinese',
            'russian',
            'north-american',            
            'south-american',
            'american',
            'african',
            'european',
            'soup',
            'healthy',
            'seafood',
            'meat',
            'pork',
            'beef',
            'poultry', 
            'side-dishes', 
            'vegetarian', 
            'mexican', 
            'breakfast', 
            'pasta', 
            'pizza',
            'baking',
            'pies',
            'savory-pies',
            'dessert',
            'chocolate',
            'ice-cream',
            'cocktails',
            'nuts',
            'fruit',
            'vegetables',
            'eggs-dairy',
            'breads',
            'salads',
            'tea',
            'raw',
            'dietary',
            'crock-pot-slow-cooker',
            'oven',
            'stove-top',
            'microwave',
            '5-ingredients-or-less',
            '3-steps-or-less',
            '15-minutes-or-less',
            'main-dish',
            'dinner-party',
            'potluck',
            'heirloom-historical',
            'holiday-event'
            ]


In [160]:
fix_list = ['raw',
            'dietary',
            'savory-pies',
            'dessert',
            'salads',
            'tea']

In [161]:
tags_to_csv(fix_list)

#### Using Function

In [103]:
def tags_to_csv(tags_list):
    """
    Write CSVs of recipes based on one of its tags.

    Parameters: 
        
        tags_list — a list containing strings that correspond to recipe tags (rdf['tags']).

    Returns:
        
        Writes CSV files to path: data/cuisines/tag_string.csv
    """
    for i in tags_list:
        tag_recipes = rdf[rdf.tags.str.contains(i, case=False) == True]
        tag_recipes.to_csv(f'data/cuisine/{i}.csv')

In [159]:
tags_to_csv(cuisines)

In [157]:
rdf.tags.str.contains('salads', case=False).value_counts() # 5,832 recipes tagged with barbecue

False    216950
True      14686
Name: tags, dtype: int64

In [155]:
rdf[rdf.tags.str.contains('salad', case=False) == True] # checking values

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,kcal,fat,sugar,salt,protein,sat_fat,carbs
37,i can t believe it s low fat creamy potato salad,33165,30.0,40493,2002-07-04,"['30-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'preparation', 'salads', 'potatoes', 'vegetables', 'dietary']","[152.5, 2.0, 20.0, 13.0, 7.0, 3.0, 10.0]",12,"['cut potatoes into 1 / 2"" pieces', 'place in medium saucepan', 'add water to cover', 'bring to a boil', 'cover , reduce heat , and simmer 15-20 minutes or until tender', 'drain and cool', 'combin...",the name says it all. you have to try it to believe it!,"['red potatoes', 'green onion', 'diced pimentos', 'fat-free mayonnaise', 'plain low-fat yogurt', 'low-fat sour cream', 'sugar', 'prepared mustard', 'white wine vinegar', 'salt', 'pepper', 'celery ...",13,152.5,2.0,20.0,13.0,7.0,3.0,10.0
44,italian sandwich pasta salad,33606,25.0,26278,2002-07-08,"['30-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'preparation', 'occasion', 'main-dish', 'salads', 'pasta', 'easy', 'kid-friendly', 'summer', 'dietary', 'one-dish-meal', 'season...","[279.2, 14.0, 12.0, 22.0, 23.0, 11.0, 12.0]",6,"['cook pasta and set aside', 'place onions , dill pickles , olives , water chestnuts , chives and peppers in a bowl', 'add oil , vinegar and spices', 'marinate for 15-20 minutes or so', 'add pasta...",this is a fun salad that uses the same ingredients you would in an italian sandwich. water chestnuts give it a nice crunch!!,"['tri-color spiral pasta', 'dill pickles', 'ripe olives', 'green onion', 'chives', 'sweet pepper', 'water chestnut', 'tomatoes', 'ham', 'cheese', 'olive oil', 'cider vinegar', 'onion powder', 'gar...",16,279.2,14.0,12.0,22.0,23.0,11.0,12.0
65,munch without guilt tomatoes,30300,10.0,6164,2002-06-04,"['15-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'low-protein', 'healthy', '5-ingredients-or-less', 'appetizers', 'salads', 'vegetables', '...","[3.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",3,"['put a slice of tomato on each biscuit / cracker', 'smear with little mayo and sprinkle black pepper', 'munch without guilt !']",anytime munchies...another sweat free preparation for all of you !,"['tomatoes', 'crackers', 'mayonnaise', 'black pepper']",4,3.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231557,zucchini tomato salad,245397,195.0,294953,2007-08-08,"['time-to-make', 'course', 'preparation', 'low-protein', 'healthy', 'salads', 'side-dishes', 'dietary', 'low-sodium', 'low-cholesterol', 'low-calorie', 'low-carb', 'healthy-2', 'low-in-something',...","[217.3, 31.0, 15.0, 7.0, 4.0, 14.0, 2.0]",6,"['wash zucchini and cut in very thin slices', 'combine with tomatoes , green pepper , green onions and parsley in a large bowl', 'mix the remaining ingredients except lettuce in another bowl', 'po...",this recipe came from my fil's girlfriend. she sent me a bunch of zucchini recipes since that wonderful veggie always over-produces. enjoy. please feel free to tweak to your personal tastes.,"['zucchini', 'tomatoes', 'green pepper', 'green onion', 'fresh parsley', 'vinegar', 'salt', 'garlic salt', 'black pepper', 'salad oil', 'lettuce']",11,217.3,31.0,15.0,7.0,4.0,14.0,2.0
231577,zupas summer chicken salad,192793,10.0,250920,2006-10-30,"['15-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'for-1-or-2', 'lunch', 'salads', 'fruit', 'vegetables', 'american', 'eas...","[2954.6, 350.0, 806.0, 54.0, 31.0, 91.0, 74.0]",8,"['in a medium sized bowl toss the mixed greens , dressing , and chicken', 'transfer into serving bowl', 'top salad with dried cranberries , red grapes , and sliced strawberries', 'sprinkle chopped...",this recipe is from a restaurant in the area called zupas that i just learned about today. they shared two of their recipes with the public and th

In [104]:
tag_list = ['pastry', 'swedish']
tags_to_csv(tag_list)

In [107]:
pd.read_csv('data/cuisine/swedish.csv')

,Unnamed: 0,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,kcal,fat,sugar,salt,protein,sat_fat,carbs
0,140,rtsoppa swedish yellow pea soup,428056,150.0,618715,2010-06-01,"['time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'soups-stews', 'vegetables', 'scandinavian', 'european', 'swedish', '4-hours-or-less']","[33.2, 0.0, 12.0, 25.0, 1.0, 0.0, 2.0]",14,"['rinse and pick through the peas', ""if using whole peas , allow to sit in the water you'll use for your soup overnight"", ""this is unnecessary if you're using split peas as long as you simmer the ...","according to the site where i found this recipe, this soup is traditionally served on thursdays and is usually followed by crepe-like pancakes and whipped cream and preserves for dessert. a teaspo...","['dried yellow peas', 'water', 'onions', 'clove', 'carrot', 'ham bone', 'dried thyme', 'ground ginger', 'salt', 'pepper', 'brown mustard']",11,33.2,0.0,12.0,25.0,1.0,0.0,2.0
1,2283,absolut wonder,251020,4.0,383346,2007-09-04,"['15-minutes-or-less', 'time-to-make', 'course', 'cuisine', 'preparation', 'for-1-or-2', '5-ingredients-or-less', 'beverages', 'scandinavian', 'easy', 'european', 'swedish', 'cocktails', 'number-o...","[192.7, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",4,"['rim the glass in lemon juice then in chocolate grated', 'pour vodka and liqueur in a shaker cold', 'shake , then strain 2 times in a cocktail glass', 'let drop a marashino cherry in the bottom o...",a drink from the bar absolut ice bar at jukkasjarvi in sweden. taken in a book.,"['vanilla vodka', 'white chocolate liqueur']",2,192.7,0.0,0.0,0.0,0.0,0.0,0.0
2,4123,almost famous swedish meatballs,466324,50.0,264745,2011-10-17,"['weeknight', '60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'appetizers', 'beef', 'pork', 'scandinavian', 'european', 'dinner-party', 'he...","[310.3, 31.0, 5.0, 7.0, 33.0, 50.0, 4.0]",21,"['make the meatballs: put the breadcrumbs in a large bowl', 'heat the butter in a skillet over medium heat', 'add the onion , garlic , allspice , 2 teaspoons salt and 1 / 4 teaspoons white pepper ...",love these as appetizers! from food network magazine.,"['breadcrumbs', 'unsalted butter', 'white onion', 'garlic cloves', 'ground allspice', 'kosher salt', 'fresh ground white pepper', 'milk', 'worcestershire sauce', 'lean ground beef', 'ground lean p...",19,310.3,31.0,5.0,7.0,33.0,50.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,226535,white pepper cookies,170267,80.0,288218,2006-05-29,"['time-to-make', 'course', 'cuisine', 'preparation', 'for-large-groups', 'hand-formed-cookies', 'desserts', 'scandinavian', 'european', 'cookies-and-brownies', 'swedish', 'number-of-servings', '4-...","[123.7, 6.0, 35.0, 4.0, 3.0, 12.0, 6.0]",6,"['with an electric mixer beat the sugar with the butter', 'then add one egg , a cup of flour , one egg , one cup of flour , the spices and at last the rest of the flour', 'make a soft dough into a...",a friend of mine has a mother in law from sweden. i ask her to give me the recipe for her wonderful cookies.post for world tour 2006.,"['all-purpose flour', 'butter', 'brown sugar', 'eggs', 'cinnamon', 'white pepper', 'baking powder', 'baking soda', 'ground ginger', 'cardamom', 'salt']",11,123.7,6.0,35.0,4.0,3.0,12.0,6.0
371,226790,whole grain swedish rye bread,206578,270.0,248918,2007-01-21,"['course', 'main-ingredient', 'cuisine', 'preparation', 'for-large-groups', 'healthy', 'breads', 'scandinavian', 'european', 'swedish', 'grains', 'dietary', 'low-sodium', 'low-cholesterol', 'healt...","[123.0, 2.0, 20.0, 0.0, 7.0, 3.0, 8.0]",11,"['combine honey , molasses , salt , butter and caraway , pour boiling water over , and set aside to cool', 'add yeast to warm water , stir to dissolve , and add to honey molasses mixture when cool...","i love rye bread, but try to avoid breads with white flour. i got this recipe

In [151]:
rdf[(rdf.tags.str.contains('bread', case=False) == True) & (rdf.tags.str.contains('breads',case=False) == False)]

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,kcal,fat,sugar,salt,protein,sat_fat,carbs
2264,abm perfect pizza dough,195526,65.0,20480,2006-11-14,"['time-to-make', 'course', 'cuisine', 'preparation', 'north-american', 'healthy', 'main-dish', 'bread-machine', 'easy', 'pizza', 'dietary', 'low-cholesterol', 'low-saturated-fat', 'healthy-2', 'lo...","[260.5, 7.0, 5.0, 18.0, 15.0, 4.0, 15.0]",3,"['follow the instructions to make dough in your bread machine', 'this dough can be used now or cut in 2', 'store in freezer for 2 pizzas or 2 calzones']",this is the easiest to make and roll pizza/calzone dough.you can keep it in the freezer for when you get the urge to make a pizza or calzone.,"['water', 'margarine', 'salt', 'ground black pepper', 'sugar', 'white flour', 'whole wheat flour', 'yeast']",8,260.5,7.0,5.0,18.0,15.0,4.0,15.0
4713,amazing stuffing from scratch breadmaker recommended,338907,105.0,186578,2008-11-21,"['time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'stuffings-dressings', 'side-dishes', 'vegetables', 'american', 'oven', 'bread-machine', 'holi...","[543.9, 45.0, 19.0, 65.0, 30.0, 74.0, 19.0]",45,"[""for the bread- set up your breadmaker for a 2 pound loaf on the wheat setting and follow your breadmaker's instructions on what order to add ingredients if it's different from my order"", 'combin...","this is the best stuffing ever. even my picky inlaws love it. \r\n\r\nthis recipe takes some planning but all of the prep work can be done well ahead of time. i make the bread, then combine all th...","['milk', 'salt', 'butter', 'white flour', 'whole wheat flour', 'sugar', 'active dry yeast', 'fresh coarse ground black pepper', 'thyme', 'rosemary', 'sage', 'savory', 'parsley', 'poultry seasoning...",21,543.9,45.0,19.0,65.0,30.0,74.0,19.0
6321,apple butter bread machine,147791,85.0,50509,2005-12-08,"['time-to-make', 'course', 'main-ingredient', 'preparation', 'jams-and-preserves', 'condiments-etc', 'fruit', 'bread-machine', 'easy', 'beginner-cook', 'vegan', 'vegetarian', 'dietary', 'inexpensi...","[509.4, 0.0, 350.0, 2.0, 0.0, 0.0, 44.0]",3,"['place all ingredients in bread pan', 'set to jam cycle and start', 'after cycle is finished , pour into containers and refrigerate']",posted in response to a request. from a betty crocker book. i have not tried this.,"['applesauce', 'sugar', 'brown sugar', 'powdered fruit pectin', 'ground cinnamon', 'ground allspice', 'ground cloves']",7,509.4,0.0,350.0,2.0,0.0,0.0,44.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225654,wheat pizza crust bread machine,85623,75.0,116469,2004-03-03,"['time-to-make', 'course', 'cuisine', 'preparation', 'occasion', 'north-american', 'healthy', 'main-dish', 'american', 'bread-machine', 'pizza', 'dietary', 'low-cholesterol', 'low-saturated-fat', ...","[817.8, 24.0, 27.0, 73.0, 45.0, 11.0, 49.0]",8,"['in bread machine pan , place the ingredients in order suggested by manufacturer', 'select dough setting', 'when the cycle is complete , turn dough onto lightly floured surface', 'punch down , co...",this recipe comes from the may/june 2000 issue of,"['water', 'olive oil', 'sugar', 'salt', 'dried oregano', 'dried basil', 'garlic powder', 'all-purpose flour', 'whole wheat flour', 'active dry yeast']",10,817.8,24.0,27.0,73.0,45.0,11.0,49.0
227066,whole wheat pizza dough bread machine,237952,15.0,483149,2007-06-30,"['15-minutes-or-less', 'time-to-make', 'course', 'preparation', 'main-dish', 'bread-machine', 'easy', 'pizza', 'equipment', 'small-appliance', '3-steps-or-less']","[129.7, 3.0, 0.0, 12.0, 8.0, 1.0, 8.0]",3,"['place ingredients , in order listed , in bread pan fitted with kneading paddle', 'select dough / pizza dough cycle', 'when completed , deflate dough and use to make your favorite pizza']",healthy option for homemade pizza!,"['water', 'salt', 'extra virgin olive oil', 'bread flour', 'whole wheat flour', 

### Vegan

In [25]:
# rdf.tags.str.find('vegan').value_counts()
# rdf[rdf.tags.str.find('vegan') == -1]
rdf.tags.str.contains('vegan', case=False).value_counts() # 10,012 recipes tagged with vegan

False    221624
True      10012
Name: tags, dtype: int64

In [21]:
rdf[rdf.tags.str.contains('vegan', case=False) == True] # checking values

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
6,aww marinated olives,25274,15,21730,2002-04-14,"['15-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'appetizers', 'fruit', 'canadian', 'dinner-party', 'vegan', 'vegetarian'...","[380.7, 53.0, 7.0, 24.0, 6.0, 24.0, 6.0]",4,"['toast the fennel seeds and lightly crush them', 'place all the ingredients in a bowl , stir well', 'cover and leave to marinate', 'keep refrigerated and use within 1 to 2 days']",my italian mil was thoroughly impressed by my non-italian treatment of her olives. they are great appetizers and condiments to your fav pasta.(from the vancouver sun) ps. cook time include fridge ...,"['fennel seeds', 'green olives', 'ripe olives', 'garlic', 'peppercorn', 'orange rind', 'orange juice', 'red chile', 'extra virgin olive oil']",9
19,cream of cauliflower soup vegan,23850,110,3288,2002-03-28,"['lactose', 'weeknight', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'bisques-cream-soups', 'soups-stews', 'potatoes', 'vegetables', 'ameri...","[174.2, 4.0, 24.0, 1.0, 15.0, 1.0, 10.0]",10,"['heat the oil or margarine in a soup pot and add the onion and garlic', 'saute over medium heat until the onion is golden , about 10 minutes', 'add the cauliflower , potatoes , water , and season...",this is a dairy free,"['canola oil', 'onion', 'garlic', 'cauliflower', 'potatoes', 'vegetable bouillon cubes', 'water', 'salt free herb and spice seasoning mix', 'ground coriander', 'great northern bean', 'salt and pep...",16
21,cream of spinach soup vegan,24701,55,3288,2002-04-08,"['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'preparation', 'occasion', 'bisques-cream-soups', 'main-dish', 'soups-stews', 'vegetables', 'vegan', 'vegetarian', 'stove-top', ...","[64.8, 3.0, 13.0, 54.0, 4.0, 2.0, 3.0]",10,"['in a 3 qt saucepan over medium high heat , saute the onions and scallions in the apple juice and oil for 5 minutes , stirring frequently', 'add the spinach , parsley and celery', 'cook for 5-7 m...","thickened with a mix of cooked oats and vegies, this soup has all the flavor of the original with a fraction of the fat stuff. low in cholestorol too!","['onion', 'scallion', 'apple juice', 'olive oil', 'spinach', 'fresh parsley', 'celery', 'broth', 'rolled oats', 'salt', 'dried thyme', 'white pepper']",12
...,...,...,...,...,...,...,...,...,...,...,...,...
231572,zulu cabbage,455000,40,1058097,2011-05-04,"['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'low-protein', 'side-dishes', 'vegetables', 'african', 'vegan', 'vegetarian', 'dietary', 'low-sodium',...","[94.0, 7.0, 25.0, 4.0, 4.0, 3.0, 4.0]",5,"['heat the oil in a large skillet', 'add onion and pepper , and saute until softened and onion is translucent', 'add cabbage , tomatoes , curry powder , tomatoes with juice , salt and pepper', 'co...",adapted from the african kitchen.,"['sunflower oil', 'onion', 'green bell pepper', 'head white cabbage', 'canned tomatoes', 'mild curry powder', 'salt', 'pepper', 'water']",9
231610,zurie s overnight no knead bread,196201,70,200862,2006-11-18,"['time-to-make', 'course', 'main-ingredient', 'preparation', 'healthy', '5-ingredients-or-less', 'breads', 'easy', 'low-fat', 'vegan', 'vegetarian', 'grains', 'dietary', 'low-cholesterol', 'low-sa...","[1379.6, 5.0, 12.0, 194.0, 79.0, 3.0, 96.0]",82,"[""since first making this bread according to the original instructions , i've simplified the process"", 'i have now bought a fairly deep plastic bowl with sloping edges , so it has a smallish botto...","after posting this recipe, and making the bread at least 3 times a week, i have simplified the rising process, and eliminated the messy parts. the original recipe comes from baker jim lahey of the...","['white bread flour', 'instant yeast', 'sugar', 'salt', 'water']",5
2

In [29]:
vegan_recipes = rdf[rdf.tags.str.contains('vegan', case=False) == True] # checking values
vegan_recipes.to_csv('data/cuisine/vegan.csv')

In [30]:
pd.read_csv('data/cuisine/vegan.csv', index_col = 'Unnamed: 0') # confirming successful writing of CSV

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,kcal,fat,sugar,salt,protein,sat_fat,carbs
6,aww marinated olives,25274,15.0,21730,2002-04-14,"['15-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'appetizers', 'fruit', 'canadian', 'dinner-party', 'vegan', 'vegetarian'...","[380.7, 53.0, 7.0, 24.0, 6.0, 24.0, 6.0]",4,"['toast the fennel seeds and lightly crush them', 'place all the ingredients in a bowl , stir well', 'cover and leave to marinate', 'keep refrigerated and use within 1 to 2 days']",my italian mil was thoroughly impressed by my non-italian treatment of her olives. they are great appetizers and condiments to your fav pasta.(from the vancouver sun) ps. cook time include fridge ...,"['fennel seeds', 'green olives', 'ripe olives', 'garlic', 'peppercorn', 'orange rind', 'orange juice', 'red chile', 'extra virgin olive oil']",9,380.7,53.0,7.0,24.0,6.0,24.0,6.0
19,cream of cauliflower soup vegan,23850,110.0,3288,2002-03-28,"['lactose', 'weeknight', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'bisques-cream-soups', 'soups-stews', 'potatoes', 'vegetables', 'ameri...","[174.2, 4.0, 24.0, 1.0, 15.0, 1.0, 10.0]",10,"['heat the oil or margarine in a soup pot and add the onion and garlic', 'saute over medium heat until the onion is golden , about 10 minutes', 'add the cauliflower , potatoes , water , and season...",this is a dairy free,"['canola oil', 'onion', 'garlic', 'cauliflower', 'potatoes', 'vegetable bouillon cubes', 'water', 'salt free herb and spice seasoning mix', 'ground coriander', 'great northern bean', 'salt and pep...",16,174.2,4.0,24.0,1.0,15.0,1.0,10.0
21,cream of spinach soup vegan,24701,55.0,3288,2002-04-08,"['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'preparation', 'occasion', 'bisques-cream-soups', 'main-dish', 'soups-stews', 'vegetables', 'vegan', 'vegetarian', 'stove-top', ...","[64.8, 3.0, 13.0, 54.0, 4.0, 2.0, 3.0]",10,"['in a 3 qt saucepan over medium high heat , saute the onions and scallions in the apple juice and oil for 5 minutes , stirring frequently', 'add the spinach , parsley and celery', 'cook for 5-7 m...","thickened with a mix of cooked oats and vegies, this soup has all the flavor of the original with a fraction of the fat stuff. low in cholestorol too!","['onion', 'scallion', 'apple juice', 'olive oil', 'spinach', 'fresh parsley', 'celery', 'broth', 'rolled oats', 'salt', 'dried thyme', 'white pepper']",12,64.8,3.0,13.0,54.0,4.0,2.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231572,zulu cabbage,455000,40.0,1058097,2011-05-04,"['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'low-protein', 'side-dishes', 'vegetables', 'african', 'vegan', 'vegetarian', 'dietary', 'low-sodium',...","[94.0, 7.0, 25.0, 4.0, 4.0, 3.0, 4.0]",5,"['heat the oil in a large skillet', 'add onion and pepper , and saute until softened and onion is translucent', 'add cabbage , tomatoes , curry powder , tomatoes with juice , salt and pepper', 'co...",adapted from the african kitchen.,"['sunflower oil', 'onion', 'green bell pepper', 'head white cabbage', 'canned tomatoes', 'mild curry powder', 'salt', 'pepper', 'water']",9,94.0,7.0,25.0,4.0,4.0,3.0,4.0
231610,zurie s overnight no knead bread,196201,70.0,200862,2006-11-18,"['time-to-make', 'course', 'main-ingredient', 'preparation', 'healthy', '5-ingredients-or-less', 'breads', 'easy', 'low-fat', 'vegan', 'vegetarian', 'grains', 'dietary', 'low-cholesterol', 'low-sa...","[1379.6, 5.0, 12.0, 194.0, 79.0, 3.0, 96.0]",82,"[""since first making this bread according to the original instructions , i've simplified the process"", 'i have now bought a fairly deep plastic bowl with sloping edges , so it has a smallish botto...","after posting this recipe, and making the bread at least 3 times

### Barbecue

In [54]:
rdf.tags.str.contains('barbecue', case=False).value_counts() # 4,116 recipes tagged with barbecue

False    227520
True       4116
Name: tags, dtype: int64

In [55]:
rdf[rdf.tags.str.contains('barbecue', case=False) == True] # checking values

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,kcal,fat,sugar,salt,protein,sat_fat,carbs
62,momma s special marinade,30131,10.0,41480,2002-06-03,"['15-minutes-or-less', 'time-to-make', 'course', 'cuisine', 'preparation', 'occasion', 'north-american', 'for-1-or-2', 'low-protein', 'condiments-etc', 'american', 'barbecue', 'easy', 'diabetic', ...","[199.2, 31.0, 6.0, 10.0, 1.0, 13.0, 1.0]",4,"['depending on how much meat i have i usually have to either double or triple this recipe the longer your meat marinates the better it tastes', 'this recipe was originally for salmon , but i think...",we usually use with chicken. my neices ask for this by name,"['lemon juice', 'oil', 'worcestershire sauce', 'basil', 'soy sauce', 'garlic powder', 'pepper']",7,199.2,31.0,6.0,10.0,1.0,13.0,1.0
99,the woiks dilly burgers,34930,32.0,23302,2002-07-24,"['bacon', '60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'lunch', 'main-dish', 'beef', 'eggs-dairy', 'fruit', 'pork', 'v...","[449.2, 50.0, 16.0, 19.0, 58.0, 67.0, 2.0]",8,"['in a bowl , mix together the ground beef , montreal steak seasoning , bacon crumbles and dill weed', 'shape into four burgers', 'brush or spray the onion slices with olive oil', 'place the burge...",NaN,"[""mccormick's montreal brand steak seasoning"", 'dill weed', 'no-salt-added ketchup', 'ground sirloin', 'bacon', 'american cheese', 'iceberg lettuce', 'tomatoes', 'onion']",9,449.2,50.0,16.0,19.0,58.0,67.0,2.0
121,1 00 tangy chicken recipe,42508,280.0,18660,2002-10-07,"['weeknight', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'main-dish', 'poultry', 'american', 'southern-united-states', 'barbecue', 'potluc...","[969.6, 93.0, 55.0, 83.0, 148.0, 86.0, 7.0]",8,"['combine vinegar , worcestershire , chili powder , salt , pepper , and hot pepper sauce in a small bowl', ""you'll have mixed all ingredients except bbq sauce"", 'place chicken pieces in a large pl...","this is one of our all time favorite chicken bbq recipes. the recipe produces a spicy moist, flavorful bbq chicken. you might say its one of our signature bbq recipes. the idea for the chicken cam...","['chicken', 'cider vinegar', 'worcestershire sauce', 'chili powder', 'salt', 'black pepper', 'hot red pepper sauce', 'barbecue sauce']",8,969.6,93.0,55.0,83.0,148.0,86.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230962,zucchini and squash on the grill,67277,25.0,69838,2003-07-24,"['30-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'preparation', 'low-protein', 'healthy', 'side-dishes', 'vegetables', 'barbecue', 'dietary', 'low-sodium', 'low-cholesterol', 'l...","[176.9, 21.0, 13.0, 0.0, 6.0, 9.0, 4.0]",6,"['heat your grill to medium', 'toss veggies with olive oil in a large bowl', 'add salt , pepper , and cayenne and toss again', 'grill until tender , approximately 6-8 minutes per side', 'return th...","these veggies are delicious! the cayenne realllly adds something to these veggies, so try not to leave it out. if you have to, its ok :-)","['zucchini', 'yellow squash', 'yellow bell peppers', 'olive oil', 'salt and pepper', 'cayenne pepper', 'fresh basil']",7,176.9,21.0,13.0,0.0,6.0,9.0,4.0
231059,zucchini burgers no not vegetarian,65086,25.0,21730,2003-06-19,"['30-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'main-dish', 'beef', 'vegetables', 'canadian', 'barbecue', 'easy', 'holi...","[350.3, 21.0, 24.0, 12.0, 56.0, 25.0, 8.0]",3,"['mix first six ingredients together , add salt and pepper to taste , and form four patties', 'grill until desired doneness is reached , placing one slice of cheese on each burger when they are tu...",yum. one of the top burgers i've ever made. get grilling!,"['lean ground beef', 'zucchini', 'onion', 'dill weed', 'dry mustard',

In [56]:
bbq_recipes = rdf[rdf.tags.str.contains('barbecue', case=False) == True]
bbq_recipes.to_csv('data/cuisine/barbecue.csv')

### Japanese

In [36]:
rdf.tags.str.contains('japanese', case=False).value_counts() # 851 recipes tagged with Japanese

False    230785
True        851
Name: tags, dtype: int64

In [37]:
rdf[rdf.tags.str.contains('japanese', case=False) == True] # checking values

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,kcal,fat,sugar,salt,protein,sat_fat,carbs
412,hibachi style salad dressing,170199,15.0,282673,2006-05-29,"['15-minutes-or-less', 'time-to-make', 'course', 'cuisine', 'preparation', 'salads', 'asian', 'japanese', 'no-cook', 'salad-dressings', 'technique']","[97.2, 12.0, 8.0, 16.0, 1.0, 6.0, 1.0]",5,"['to make ginger juice , peel some fresh ginger and cut into small pieces', 'put pieces in a garlic press and squeeze', 'combine all ingredients except mayonnaise in a small bowl and whisk until h...","this is my attempt to replicate a salad dressing from a japanese restaurant i used to frequent many years ago. while this may not be authentically japanese, it sure is tasty! you can really pick u...","['rice vinegar', 'lemon juice', 'soy sauce', 'honey', 'ginger juice', 'sesame oil', 'mayonnaise']",7,97.2,12.0,8.0,16.0,1.0,6.0,1.0
502,philly roll sushi,285568,10.0,238966,2008-02-09,"['15-minutes-or-less', 'time-to-make', 'cuisine', 'preparation', 'asian', 'japanese', 'dietary', 'low-sodium', 'low-in-something']","[406.7, 32.0, 5.0, 14.0, 24.0, 63.0, 14.0]",10,"['place one full sheet of sushi nori on a bamboo rolling mat , shiny side down', 'cover the sheet lightly with the prepared sushi rice- leaving the remaining inch and a half of nori bare lengthwis...",makes 2 delicious & easy sushi rolls!,"['cream cheese', 'smoked salmon', 'english cucumber', 'nori', 'sushi rice', 'toasted sesame seeds']",6,406.7,32.0,5.0,14.0,24.0,63.0,14.0
817,10 min japanese simmered tofu hot pot,39169,10.0,54102,2002-09-03,"['15-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'for-1-or-2', 'side-dishes', 'beans', 'asian', 'japanese', 'stove-top', 'dietary', 'comfor...","[166.3, 10.0, 35.0, 43.0, 24.0, 6.0, 4.0]",7,"[""cut the tofu into edible'cubes' and roughly chop the green onions on a bias for presentation"", 'evenly spread the fish flakes in a small earthenware casserole dish / pot / dutch oven and delicat...","this dish is the epitomy of 'home style' japanese comfort food. this version is lightening quick, without the need to prepare the fish stock (dashi) that so many real japanese dishes require. this...","['soft silken tofu', 'dried fish flakes', 'soy sauce', 'sugar', 'sake', 'water', 'egg', 'green onion']",8,166.3,10.0,35.0,43.0,24.0,6.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230302,zaru soba chilled japanese noodles,448296,10.0,336467,2011-02-07,"['15-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'for-1-or-2', 'lunch', 'asian', 'japanese', 'easy', 'pasta-rice-and-grains', 'number-of-servings']","[350.5, 1.0, 6.0, 112.0, 31.0, 0.0, 25.0]",15,"['combine dashi , shoyu , mirin and vinegar in a pan and bring to a simmer', 'remove from heat and store in refrigerator to chill', 'bring litre of water to boil', 'add soba noodles', 'cook for 4 ...","this is a dish i became addicted to in thailand, of all places... i'd never seen it here in australia, so i learned to make it myself. it's extremely refreshing in hot weather. one way of servi...","['soba noodles', 'dashi', 'shoyu', 'mirin', 'rice wine vinegar', 'spring onions', 'wasabi', 'nori']",8,350.5,1.0,6.0,112.0,31.0,0.0,25.0
230330,zenzai dango in azuki,312691,150.0,247473,2008-07-08,"['time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', '5-ingredients-or-less', 'desserts', 'beans', 'asian', 'japanese', 'easy', 'kid-friendly', 'dietary', 'gluten-fr...","[449.4, 1.0, 250.0, 12.0, 6.0, 1.0, 35.0]",11,"['pick over and wash the azuki , then put them in a pan with 6 cups of water', 'bring to boil , then lower the heat and simmer for 2 hours until the beans are soft', 'add more water to the pan to ...","zenzai is the name for mochi dumplings, also known as ""dango"" made in a soup of sweet red beans (azuki). \r\n\r\nit is an ancient j

In [38]:
japanese_recipes = rdf[rdf.tags.str.contains('japanese', case=False) == True]
japanese_recipes.to_csv('data/cuisine/japanese.csv')

### Mediterranean

In [41]:
rdf.tags.str.contains('italian', case=False).value_counts() # 7,410 recipes tagged with barbecue

False    224226
True       7410
Name: tags, dtype: int64

In [42]:
rdf[rdf.tags.str.contains('italian', case=False) == True] # checking values

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,kcal,fat,sugar,salt,protein,sat_fat,carbs
51,killer lasagna,53402,90.0,60512,2003-02-04,"['weeknight', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'casseroles', 'main-dish', 'pasta', 'oven', 'european', 'potluck', 'dinner-party', 'heirloom-histor...","[370.5, 38.0, 21.0, 57.0, 53.0, 64.0, 3.0]",13,"['brown the sausage and ground meat and drain all the fat', 'add all the other ingredients for the meat filling and simmer on the stove , uncovered , for about 1 / 2 hour', 'i usually add a bay le...",this was originally my mom's recipe and used to be a favorite when we were kids. i always requested it for my birthday dinner.,"['italian sausage', 'ground beef', 'garlic', 'dried basil', 'salt', 'whole tomato', 'tomato paste', 'ricotta cheese', 'cottage cheese', 'parmesan cheese', 'parsley flakes', 'eggs', 'ground black p...",15,370.5,38.0,21.0,57.0,53.0,64.0,3.0
66,my muffuletta sandwich,78655,20.0,12875,2003-12-12,"['30-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'lunch', 'pork', 'easy', 'european', 'finger-food', 'italian', 'sandwiches', 'meat', 'nove...","[181.1, 26.0, 6.0, 17.0, 2.0, 11.0, 2.0]",3,"['mix everything in food processor', 'chop fine for spread', 'refrigerate until ready to use']",watched a documentary about the,"['ciabatta', 'provolone cheese', 'genoa salami', 'ham', 'tomatoes', 'sweet onion', 'green olives', 'kalamata olive', 'fresh basil', 'capers', 'garlic', 'brine', 'oregano leaves', 'olive oil', 'red...",17,181.1,26.0,6.0,17.0,2.0,11.0,2.0
79,real italian bolognese sauce,83950,160.0,124847,2004-02-14,"['time-to-make', 'course', 'cuisine', 'preparation', 'sauces', 'condiments-etc', 'european', 'italian', 'dietary', '4-hours-or-less']","[1260.7, 97.0, 11.0, 71.0, 103.0, 119.0, 38.0]",16,"['in a dutch oven or medium size pot , heat butter and olive oil over medium heat until butter begins to froth', 'add onion , celery , carrot , garlic , and bacon', 'cook until onions are transluc...","after traveling throughout italy, savoring the fine tastes of bolognese from the many different regions, i decided to formulate my own. try it, you'll love it.","['bacon', 'onion', 'celery', 'carrot', 'garlic', 'butter', 'olive oil', 'lean ground beef', 'ground pork', 'beef consomme', 'dry white wine', 'crushed tomatoes', 'salt', 'black pepper', 'rubbed sa...",20,1260.7,97.0,11.0,71.0,103.0,119.0,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231592,zuppa inglese della nonna,456931,26.0,1680722,2011-05-22,"['30-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'desserts', 'eggs-dairy', 'european', 'vegetarian', 'italian', 'dietary']","[393.0, 23.0, 115.0, 12.0, 19.0, 31.0, 18.0]",20,"['in a pot wish together very well egg yolks , sugar and corn flour until light', 'add lemon zest and vanilla', 'mix well', 'add boiling milk', 'now heat , stirring , until it barely reaches a boi...","this delicious light dessert is very similar to tiramisù but it is based on lighter ingredients.\r\ngrandama prepared this often after heavy meals, we always enjoyed it as my children too.\r\nsome...","['biscuits', 'egg yolks', 'sugar', 'lemon', 'vanilla', 'cornflour', 'milk', 'espresso coffee', 'maraschino cherries', 'amaretto', 'cocoa powder']",11,393.0,23.0,115.0,12.0,19.0,31.0,18.0
231595,zuppa sarda sardinian cheese and egg soup with croutons,89330,30.0,94272,2004-04-18,"['30-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'soups-stews', 'eggs-dairy', 'easy', 'european', 'vegetarian', 'italian', 'dietary', 'comf...","[733.4, 65.0, 12.0, 46.0, 71.0, 78.0, 17.0]",10,"['preheat oven to 350 degrees f', 'brush bread slices with olive oil and rub with garlic', 'place on a baking sheet and bake about 5 minutes per side , until g

In [43]:
italian_recipes = rdf[rdf.tags.str.contains('italian', case=False) == True]
italian_recipes.to_csv('data/cuisine/italian.csv')

### Burgers

In [46]:
rdf.tags.str.contains('burgers', case=False).value_counts() # 279 recipes tagged with barbecue

False    231357
True        279
Name: tags, dtype: int64

In [47]:
rdf[rdf.tags.str.contains('burgers', case=False) == True] # checking values

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,kcal,fat,sugar,salt,protein,sat_fat,carbs
329,better than a can homemade sloppy joes,513531,50.0,1783373,2014-02-21,"['weeknight', '60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'low-protein', 'healthy', 'lunch', 'main-dish', 'beef', 'am...","[291.5, 13.0, 85.0, 25.0, 13.0, 7.0, 16.0]",26,"['to start the sauce: heat a 2 1 / 2-quart nonreactive saucepan over medium heat and , when hot , add the oil', 'when the oil is hot , stir in the onion , green and red pepper , celery and garlic'...",i've had this recipe for many years and i used to make it on a regular basis when i was a newly-wed and made it often for the kids when they were itty bitty. it is a great recipe and one well wort...,"['olive oil', 'yellow onion', 'green bell pepper', 'red bell pepper', 'celery', 'garlic cloves', 'tomato puree', 'tomato paste', 'ketchup', 'cider vinegar', 'unsulphured molasses', 'worcestershire...",18,291.5,13.0,85.0,25.0,13.0,7.0,16.0
447,little kick jalapeno burgers,371549,16.0,383853,2009-05-12,"['30-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'preparation', 'occasion', 'main-dish', 'poultry', 'easy', 'summer', 'turkey', 'seasonal', 'meat', 'equipment', 'grilling', '3-s...","[265.9, 18.0, 13.0, 19.0, 44.0, 23.0, 5.0]",8,"['in a large bowl , combine the first eight ingredients', 'crumble turkey over mixture and mix well', 'shape into 6 patties', 'coat grill rack with cooking spray before starting the grill', 'grill...","this recipe was originally submitted to taste of home's healthy cooking magazine by dawn dhooghe of concord, north carolina. ""i lightened up one of my husband's favorite burgers, although the ori...","['jalapeno peppers', 'fresh cilantro', 'light beer', 'hot pepper sauce', 'garlic cloves', 'pepper', 'salt', 'cayenne pepper', 'lean ground turkey', 'monterey jack pepper cheese', 'dinner rolls', '...",14,265.9,18.0,13.0,19.0,44.0,23.0,5.0
805,1 3 pound squeeze burger,344373,25.0,23119,2008-12-20,"['30-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'preparation', 'main-dish', 'beef', 'easy', 'dietary', 'ground-beef', 'meat', '3-steps-or-less', 'burgers']","[975.7, 112.0, 21.0, 43.0, 134.0, 204.0, 3.0]",10,"[""divide the meat into 3 evenly sized burgers and seasoned with lawry's seasoned salt to taste"", 'cook the burgers to the desired doneness on a griddle or in a large cast-iron skillet', 'top the b...","adapted from a recipe courtesy of squeeze inn. from ""diners, drive-ins and dives"" cookbook. frying these on a hot griddle, the owner takes a 1/3 pound handful of cheddar cheese to top a 1/3 pound...","['ground beef', ""lawry's seasoned salt"", 'mild cheddar cheese', 'ice cube', 'sourdough hamburger buns', 'dill pickle', 'red onion', 'yellow onion', 'tomatoes', 'lettuce leaf']",10,975.7,112.0,21.0,43.0,134.0,204.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226696,whitey s oven burgers,508577,20.0,2399085,2013-10-30,"['weeknight', '30-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'preparation', 'occasion', 'main-dish', 'beef', 'inexpensive', 'ground-beef', 'meat', 'number-of-servings', 'burgers']","[170.5, 16.0, 2.0, 19.0, 26.0, 23.0, 1.0]",6,"['preheat oven to 400 degrees', 'mix all ingredients except buns , pickles , and cheese', 'put mixture in an ungreased 10x13 pan and spread evenly edge to edge', 'prick with a fork several times t...","this recipe taste like a burger joints burgers but better, they still taste great even if you are not drunk.","['ground beef', 'warm water', 'egg', 'onion', 'panko breadcrumbs', 'onion powder', 'salt', 'pepper', 'hawaiian rolls', 'kosher pickles', 'american cheese']",11,170.5,16.0,2.0,19.0,26.0,23.0,1.0
229572,you can bet the farm hamburgers from longmeadow farm,267986,20.0,428885,2007-11-27,"['30-minutes-or-less', 't

In [68]:
burger_recipes = rdf[rdf.tags.str.contains('burgers', case=False) == True]
burger_recipes.to_csv('data/cuisine/burgers.csv')

### Sandwiches

In [66]:
rdf.tags.str.contains('sandwiches', case=False).value_counts() # 5,832 recipes tagged with barbecue

False    225804
True       5832
Name: tags, dtype: int64

In [51]:
rdf[rdf.tags.str.contains('sandwiches', case=False) == True] # checking values

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,kcal,fat,sugar,salt,protein,sat_fat,carbs
43,immoral sandwich filling loose meat,58224,35.0,37183,2003-04-04,"['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'preparation', 'for-large-groups', 'very-low-carbs', 'lunch', 'main-dish', 'beef', 'easy', 'stove-top', 'dietary', 'sandwiches',...","[223.2, 22.0, 4.0, 7.0, 35.0, 30.0, 0.0]",6,"['brown the meat & drain fat', 'stir in sugar , mustard , beer , cayenne , garlic and salt & pepper to taste', 'simmer until liquid has mostly cooked away', 'lay slices of cheese over top of meat ...","just the thing for a day when you're wanton something delicious, and a hamburger patty is just too restrictive. loosen up your inhibitions and pile some loose meat on a bun. wicked over hot, whipp...","['ground beef', 'sugar', 'prepared yellow mustard', 'beer', 'cayenne', 'garlic', 'salt & pepper', 'american cheese']",8,223.2,22.0,4.0,7.0,35.0,30.0,0.0
46,italian gut busters,35173,45.0,22724,2002-07-27,"['60-minutes-or-less', 'time-to-make', 'course', 'preparation', 'lunch', 'main-dish', 'oven', 'easy', 'dietary', 'sandwiches', 'equipment']","[816.9, 100.0, 11.0, 97.0, 98.0, 180.0, 2.0]",7,"['lay out sandwich rolls on jelly roll pans / cookie sheets', 'melt butter , mix in italian dressing mix', 'using a pastry or bbq brush , graciously apply seasoned butter to the top of the""bottom ...",my sister-in-law made these for us at a family get together. they are delicious!! a little messy to make but worth the effort!! have a helper and make an,"['sandwich bun', 'good seasonings italian salad dressing mix', 'butter', 'deli turkey', 'ham', 'pepperoni', 'cheddar cheese', 'swiss cheese', 'mozzarella cheese']",9,816.9,100.0,11.0,97.0,98.0,180.0,2.0
66,my muffuletta sandwich,78655,20.0,12875,2003-12-12,"['30-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'lunch', 'pork', 'easy', 'european', 'finger-food', 'italian', 'sandwiches', 'meat', 'nove...","[181.1, 26.0, 6.0, 17.0, 2.0, 11.0, 2.0]",3,"['mix everything in food processor', 'chop fine for spread', 'refrigerate until ready to use']",watched a documentary about the,"['ciabatta', 'provolone cheese', 'genoa salami', 'ham', 'tomatoes', 'sweet onion', 'green olives', 'kalamata olive', 'fresh basil', 'capers', 'garlic', 'brine', 'oregano leaves', 'olive oil', 'red...",17,181.1,26.0,6.0,17.0,2.0,11.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230924,zucchini and cream cheese sandwich,194572,10.0,207176,2006-11-08,"['15-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'preparation', 'occasion', 'for-1-or-2', 'healthy', 'lunch', 'eggs-dairy', 'vegetables', 'easy', 'kid-friendly', 'kosher', 'low-...","[1294.5, 21.0, 26.0, 116.0, 87.0, 15.0, 82.0]",11,"['preheat grill or frying pan to hot', 'slice zucchini lengthwise into thin strips', 'brush one side of each strip with a little oil', 'lay the zucchini slices , oil side down , on the hot surface...","from ""cooking up a storm, the teen survival cookbook"" by sam stern. this is a clever creation! *variation: hummus instead of cream cheese. both sound good to me.","['zucchini', 'olive oil', 'sea salt', 'cream cheese', 'french bread', 'parsley']",6,1294.5,21.0,26.0,116.0,87.0,15.0,82.0
231429,zucchini wheat germ burgers,326351,17.0,724631,2008-09-22,"['30-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'preparation', 'lunch', 'vegetables', 'easy', 'beginner-cook', 'vegetarian', 'stove-top', 'dietary', 'sandwiches', 'squash', 'eq...","[329.3, 21.0, 16.0, 13.0, 32.0, 23.0, 11.0]",9,"['in a large mixing bowl , beat the eggs', 'stir in wheat germ , cheese , mushrooms , onion , thyme , rosemary and zucchini', 'add salt and pepper to taste', 'shape into 4 patties , 3 / 4-inch thi...","entered for safe-keeping, from sean donnellan's ""something tastes funny"" with recipes by n

In [67]:
sandwich_recipes = rdf[rdf.tags.str.contains('sandwiches', case=False) == True]
sandwich_recipes.to_csv('data/cuisine/sandwiches.csv')

# Design Notes

## Color Palette

- Red Pepper — #EA5039
- Cilantro — #74CD45
- Squash — #FFB25B
- Medium Rare — #DA9077
- Guacamaiolei — #A6E893
- Merlot — #570828
- Creme Brulee — #E69960
- Aubergine — #69494F
- Wintergreen — #c8e7a8
- Squash — #f7ae2e
- Blue Marker — #ABE2FF